In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

# import seaborn as sns
from pylab import rcParams

import psycopg2
import pandas as pd

# from conf_db import config

import os

import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

%config InlineBackend.figure_format='retina'
%load_ext nb_black
# sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams["figure.figsize"] = 16, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

import os

# os.environ["CUDA_DEVICE_ORDER"] = "GPU"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7676610008191916423
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6078012944663898101
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2779250688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16583910746041968889
physical_device_desc: "device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6891867289934168219
physical_device_desc: "device: XLA_GPU device"
]


<IPython.core.display.Javascript object>

In [2]:
from configparser import ConfigParser


def config(filename="database.ini", section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}

    # Checks to see if section (postgresql) parser exists
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]

    # Returns an error if a parameter is called that is not listed in the initialization file
    else:
        raise Exception(
            "Section {0} not found in the {1} file".format(section, filename)
        )

    return db

<IPython.core.display.Javascript object>

In [3]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    data_vstup, data_vystup = [], []
    for i in range(len(dataset) - look_back - 2):
        vstup = dataset[i : (i + look_back), 0]
        data_vstup.append(vstup)
        vystup = [dataset[i + look_back, 0], dataset[i + look_back + 1, 0]]
        data_vystup.append(vystup)
    return np.array(data_vstup), np.array(data_vystup)

<IPython.core.display.Javascript object>

In [4]:
params = config("database.ini", "warden_rady")
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params)
# Create a new cursor
cur = conn.cursor()

# A function that takes in a PostgreSQL query and outputs a pandas database
def create_pandas_table(sql_query, database=conn):
    table = pd.read_sql_query(sql_query, database)
    return table


# Utilize the create_pandas_table function to create a Pandas data frame
# Store the data as a variable
dataset_z_db = create_pandas_table(
    "SELECT time, value FROM history_int WHERE item_id = 6 ORDER BY time"
)
print(len(dataset_z_db))
print(dataset_z_db)

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

5320
                          time  value
0    2017-12-01 00:00:00+00:00      0
1    2017-12-01 01:00:00+00:00      9
2    2017-12-01 02:00:00+00:00     12
3    2017-12-01 03:00:00+00:00     18
4    2017-12-01 04:00:00+00:00     10
...                        ...    ...
5315 2018-07-10 11:00:00+00:00   1503
5316 2018-07-10 12:00:00+00:00   1610
5317 2018-07-10 13:00:00+00:00   1599
5318 2018-07-10 14:00:00+00:00   1779
5319 2018-07-10 15:00:00+00:00    827

[5320 rows x 2 columns]


<IPython.core.display.Javascript object>

In [5]:
dataframe = dataset_z_db["value"].to_frame()
dataset = dataframe.values
dataset = dataset.astype("float32")
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train_dataset, test_dataset = (
    dataset[0:train_size, :],
    dataset[train_size : len(dataset), :],
)
# reshape into X=t and Y=t+1
look_back = 10
train_vstupy, train_vystupy = create_dataset(train_dataset, look_back)
test_vstupy, test_vystupy = create_dataset(test_dataset, look_back)

print(train_vstupy.shape)
print(train_vstupy.shape[0])
print(train_vstupy.shape[1])

# reshape input to be [samples, time steps, features]
train_vstupy = np.reshape(
    train_vstupy, (train_vstupy.shape[0], train_vstupy.shape[1], 1)
)
test_vstupy = np.reshape(test_vstupy, (test_vstupy.shape[0], test_vstupy.shape[1], 1))

print(test_vstupy.shape)
print(train_vstupy.shape)
print(test_vystupy)

[[0.        ]
 [0.00051912]
 [0.00069216]
 ...
 [0.09223049]
 [0.10261291]
 [0.04770145]]
(3552, 10)
3552
10
(1744, 10, 1)
(3552, 10, 1)
[[0.30420488 0.28920805]
 [0.28920805 0.32023996]
 [0.32023996 0.28217107]
 ...
 [0.0866932  0.09286498]
 [0.09286498 0.09223049]
 [0.09223049 0.10261291]]


<IPython.core.display.Javascript object>

In [6]:
model = keras.Sequential()
model.add(
    keras.layers.LSTM(
        units=128, input_shape=(train_vstupy.shape[1], train_vstupy.shape[2])
    )
)
model.add(keras.layers.Dense(units=2))
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(0.0001))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 66,818
Trainable params: 66,818
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [7]:
history = model.fit(
    train_vstupy,
    train_vystupy,
    epochs=10,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False,
)

Train on 3196 samples, validate on 356 samples
Epoch 1/10
3196/3196 [==============================] - 5s 2ms/sample - loss: 0.0124 - val_loss: 0.0026
Epoch 2/10
3196/3196 [==============================] - 1s 275us/sample - loss: 0.0034 - val_loss: 0.0024
Epoch 3/10
3196/3196 [==============================] - 1s 266us/sample - loss: 0.0030 - val_loss: 0.0022
Epoch 4/10
3196/3196 [==============================] - 1s 273us/sample - loss: 0.0028 - val_loss: 0.0020
Epoch 5/10
3196/3196 [==============================] - 1s 271us/sample - loss: 0.0026 - val_loss: 0.0019
Epoch 6/10
3196/3196 [==============================] - 1s 270us/sample - loss: 0.0025 - val_loss: 0.0018
Epoch 7/10
3196/3196 [==============================] - 1s 263us/sample - loss: 0.0024 - val_loss: 0.0018
Epoch 8/10
3196/3196 [==============================] - 1s 259us/sample - loss: 0.0023 - val_loss: 0.0017
Epoch 9/10
3196/3196 [==============================] - 1s 265us/sample - loss: 0.0023 - val_loss: 0.0017
E

<IPython.core.display.Javascript object>

In [8]:
train_predikcia = model.predict(train_vstupy)
test_predikcia = model.predict(test_vstupy)
print(len(train_predikcia))
print(len(test_predikcia))
print(test_predikcia[:, 0])
print(test_predikcia[:, 1])
print(test_predikcia)

3552
1744
[0.27218845 0.27237597 0.27091652 ... 0.10260775 0.10132521 0.10107622]
[0.27250692 0.27202067 0.27097166 ... 0.10734701 0.10615297 0.10559522]
[[0.27218845 0.27250692]
 [0.27237597 0.27202067]
 [0.27091652 0.27097166]
 ...
 [0.10260775 0.10734701]
 [0.10132521 0.10615297]
 [0.10107622 0.10559522]]


<IPython.core.display.Javascript object>

In [ ]:
train_predikcia = scaler.inverse_transform(train_predikcia)
# trainY1 = scaler.inverse_transform(trainY[:,[1]])
test_predikcia = scaler.inverse_transform(test_predikcia)

In [16]:
print(train_predikcia[:, [0]])
print(train_predikcia)

[[ 454.8    ]
 [ 453.76675]
 [ 452.7318 ]
 ...
 [4731.8047 ]
 [4645.0425 ]
 [4546.081  ]]
[[ 454.8      557.9983 ]
 [ 453.76675  557.0631 ]
 [ 452.7318   555.9241 ]
 ...
 [4731.8047  4828.729  ]
 [4645.0425  4719.9805 ]
 [4546.081   4613.873  ]]


<IPython.core.display.Javascript object>

In [21]:
time = np.arange(0, len(dataset))

dataset_z_db = scaler.inverse_transform(dataset)
merged_list = []

for l in dataset_z_db:
    merged_list.append(int(l[0]))

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=time, y=merged_list, mode="lines", name="povodne data",))
fig.add_trace(
    go.Scatter(
        x=time[look_back : len(train_predikcia[:, [0]]) + look_back],
        y=np.reshape(train_predikcia[:, [0]], len(train_predikcia[:, [0]])),
        mode="markers",
        name="treningova mnozina",
    )
)
fig.add_trace(
    go.Scatter(
        x=time[look_back : len(train_predikcia[:, [1]]) + look_back],
        y=np.reshape(train_predikcia[:, [1]], len(train_predikcia[:, [1]])),
        mode="markers",
        name="treningova mnozina",
    )
)
fig.add_trace(
    go.Scatter(
        x=time[len(train_predikcia[:, [0]]) + (look_back * 2) + 1 : len(dataset) - 1],
        y=np.reshape(test_predikcia[:, [0]], len(test_predikcia[:, [0]])),
        mode="markers",
        name="testovacia mnozina",
    )
)
fig.add_trace(
    go.Scatter(
        x=time[len(train_predikcia[:, [1]]) + (look_back * 2) + 1 : len(dataset) - 1],
        y=np.reshape(test_predikcia[:, [1]], len(test_predikcia[:, [1]])),
        mode="markers",
        name="testovacia mnozina",
    )
)
fig.show()

<IPython.core.display.Javascript object>